# Epileptic seizure prediction based on intracranial electroencephalography (EEG) data

***
***

## Summary

### I - Feature generation

<ol>
<li> <strong><em> Setup </em></strong>  </li>
    <ol>
        <li> Imports </li>
        <li> Parameters </li>
        <li> Logging </li>
    </ol>
<br>

<li> <strong><em> Helper functions </em></strong> </li>
    <ol>
        <li> Reading data from .mat files </li>
        <li> Timeseries features </li>
        <li> Entropy features </li>
    </ol>    
<br>

<li> <strong><em> Feature generation loop </em></strong> </li>
    <ol>
        <li> Feature generation function </li>
        <li> Feature generation in loop over patients </li>
    </ol>
<br>

<li> <strong><em> EDA of the generated dataset </em></strong> </li>
    <ol>
        <li> zero values and NANs </li>
        <li> Feature distibutions </li>
    </ol>
</ol>


### II - Machine learning model

<ol>
<li> <strong><em> Setup </em></strong> </li>
    <ol>
        <li> Imports </li>
        <li> Metrics function </li>
        <li> Create test and train datasets  </li>
    </ol>
<br>

<li> <strong><em> Model </em></strong> </li>
    <ol>
        <li> Choosing type of model </li>
        <li> Hyperparameter tuning with genetic algo </li>
        <li> ROC Curve and choosing threshold </li>
    </ol>  
</ol>

***

# Abstract

Epilepsy is one of the most common brain disorders, [according to the CDC](http://www.ur.ac.rw) 1.2% of the population has active epilepsy. About 40% of them have seizures that are not controlled by medication ([Kwan and Brodie 2000](https://pubmed.ncbi.nlm.nih.gov/11034869/)). The unpredictable nature of epileptic seizures can have a significant impact on their lives, making certain common activities such as driving or swimming potentially life-threatening. Being able to predict the onset of seizures could drastically improve the quality of life of those affected.  
The point of this work is to contribute to the [collective effort](https://academic.oup.com/brain/article/141/9/2619/5066003) guided by [the Epilepsy Ecosystem](epilepsyecosystem.org).  
A classifier will be designed and trained to maximise the AUC for 3 patients. It will be trained on features generated from the 16 channel EEG data from the NeuroVista trials for these same patients and then evaluated independently by the Epilepsy Ecosystem.

# I - Feature generation

## Setup

### Imports

In [3]:
from tqdm import tqdm
import sklearn.preprocessing as preprocessing
import numpy as np
from scipy.integrate import simps
import scipy.io as sio
import scipy.stats
import scipy.signal
from os import listdir
from os.path import isfile, join
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')
import logging

### Parameters

In [4]:
DATA_PATH = 'C:/Users/gijsb/OneDrive/Documents/epilepsy_neurovista_data/'
TRAIN_PATHS = [f'Pat{i}Train' for i in [1, 2, 3]]
TEST_PATHS = [f'Pat{i}Test' for i in [1, 2, 3]]
FEATURE_SAVE_PATH = DATA_PATH # Path where output feature arrays will be saved

SAMPLING_FREQUENCY = 400
DOWNSAMPLING_RATIO = 5
CHANNELS = range(0,16)
BANDS = [0.1,1,4,8,12,30,70]
HIGHRES_BANDS = [0.1,1,4,8,12,30,70,180]

### Logging

In [5]:
#today_string = str(datetime.now())[0:19].replace('-', '_').replace(':', '_').replace(' ', '_')
now = datetime.now()
today_string = now.strftime("%d_%m_%Y__%H_%M_%S")
log_filename = f'feature_generation_log_{today_string}.log'
logging.basicConfig(level=logging.DEBUG, 
                    filename=log_filename, 
                    format='%(asctime)s.%(msecs)03d %(levelname)s {%(module)s} [%(funcName)s] %(message)s', 
                    datefmt='%Y-%m-%d,%H:%M:%S')

## Helper functions

#### Reading .mat files

In [6]:
def load_mat(mat_file_path):
    """
    input : filepath string
    output : numpy array, returns zeros if it cannot read the file
    """
    try:
        data = (sio.loadmat(mat_file_path)['data']).T
        logging.debug('data loaded')
        return data

    except Exception:
        warnings.warn(f'error reading .mat file {mat_file_path}')
        return np.zeros((16, 240000))

#### Features on timeseries

In [7]:
def zero_crossings(data):
    pos = data > 0
    return (pos[:-1] & ~pos[1:]).nonzero()[0].shape[0]


def band_energy(f, psd, low_f, high_f):
    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(f >= low_f, f <= high_f)
    # The frequency resolution is the size of each frequency bin
    freq_res = f[1] - f[0]
    # Compute the absolute power by approximating the integral
    return simps(psd[idx_delta], dx=freq_res)


def total_energy(segment_downsampled):
    # From litterature the lowest frequencies of interest in a EEG is 0.5Hz so we need to keep our resolution at 0.25Hz hence a 4 second window cf.Nyquist
    window = SAMPLING_FREQUENCY / DOWNSAMPLING_RATIO * 4
    f, psd = scipy.signal.welch(
        segment_downsampled, fs=SAMPLING_FREQUENCY/DOWNSAMPLING_RATIO, nperseg=window)
    return psd.sum()


def highres_total_energy(segment):
    window = SAMPLING_FREQUENCY * 4
    f, psd = scipy.signal.wel

#### SVD Entropy

In [8]:
def _embed(x, order=3, delay=1):  # credits to raphaelvallat
    """Time-delay embedding.
    Parameters
    ----------
    x : 1d-array
        Time series, of shape (n_times)
    order : int
        Embedding dimension (order).
    delay : int
        Delay.
    Returns
    -------
    embedded : ndarray
        Embedded time-series, of shape (n_times - (order - 1) * delay, order)
    """
    N = len(x)
    if order * delay > N:
        raise ValueError("Error: order * delay should be lower than x.size")
    if delay < 1:
        raise ValueError("Delay has to be at least 1.")
    if order < 2:
        raise ValueError("Order has to be at least 2.")
    Y = np.zeros((order, N - (order - 1) * delay))
    for i in range(order):
        Y[i] = x[i * delay:i * delay + Y.shape[1]]
    return Y.T


def svd_entropy(x, order=3, delay=1, normalize=False):
    x = np.array(x)
    mat = _embed(x, order=order, delay=delay)
    W = np.linalg.svd(mat, compute_uv=False)
    # Normalize the singular values
    W /= sum(W)
    svd_e = -np.multiply(W, np.log2(W)).sum()
    if normalize:
        svd_e /= np.log2(order)
    return svd_e

## Feature generation loop
N.B. This loop assumes that data is stored in different folders for each patient and for train and test sets (this is how the data from Seer is provided so i made use of it)

In [9]:
def generate_features(patient_number, data_path, is_training_data, save_to_disk = True):
    
    filenames = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    filelist = [join(data_path, f) for f in listdir(data_path) if isfile(join(data_path, f))]


    logging.debug(f'generated filelist of length {len(filelist)} for patient {patient_number}; is_training_data = {is_training_data}')
    
    counter = 0
    for filename in tqdm(filenames):
        
        # Lists that will contain feature names and values, we will stack these to make X_train
        index = []
        features = []

        #Load file & normalise
        data = load_mat(join(data_path, filename))
        data = preprocessing.scale(data, axis=1, with_std=True)
        data_downsampled = scipy.signal.decimate(data, 5, zero_phase=True)
        
        logging.debug(f'starting feature generation file:{counter}')
        
        # ID features
        index.append('Patient')
        features.append(patient_number)
        
        index.append('filenumber')
        features.append(filename[filename.find('_')+1:-6])
        
        #accross channels features on full data
        correlation_matrix = np.corrcoef(data)
        correlation_matrix = np.nan_to_num(correlation_matrix)
        # take only values in upper triangle to avoid redundancy
        triup_index = np.triu_indices(16, k=1)
        for i, j in zip(triup_index[0], triup_index[1]):
            features.append(correlation_matrix[i][j])
            index.append(f'correlation_{i}-{j}')

        eigenvals = np.linalg.eigvals(correlation_matrix)
        eigenvals = np.nan_to_num(eigenvals)
        eigenvals = np.real(eigenvals)
        for i in CHANNELS:
            features.append(eigenvals[i])
            index.append(f'eigenval_{i}')
            
        # summed across all channels and frequencies
        summed_energy = total_energy(data_downsampled)
        features.append(summed_energy)
        index.append('summed_energy')
        
        logging.debug('general features generated')
        
        #Per channel features
        #TODO work on all channels in parrallel as one matrix, vectorise all of it
        for c in CHANNELS:
            
            logging.debug(f'starting feature generation file:{counter}, channel:{c}')
            
            # Create necessary functions
            data_channel = data_downsampled[c]
            diff1 = np.diff(data_channel, n=1)
            diff2 = np.diff(data_channel, n=2)

            ## Simple features
            std = np.std(data_channel)
            features.append(std)
            index.append(f'std_{c}')

            skew = scipy.stats.skew(data_channel)
            features.append(skew)
            index.append(f'skew_{c}')

            kurt = scipy.stats.kurtosis(data_channel)
            features.append(kurt)
            index.append(f'kurt_{c}')

            zeros = zero_crossings(data_channel)
            features.append(zeros)
            index.append(f'zeros_{c}')
            
            logging.debug('simple features generated')

            #RMS = np.sqrt(data_channel**2.mean())

            ## Differential features
            mobility = np.std(diff1)/np.std(data_channel)
            features.append(mobility)
            index.append(f'mobility_{c}')

            complexity = (np.std(diff2) * np.std(diff2)) / np.std(diff1)
            features.append(complexity)
            index.append(f'complexity_{c}')

            zeros_diff1 = zero_crossings(diff1)
            features.append(zeros_diff1)
            index.append(f'zeros_diff1_{c}')

            zeros_diff2 = zero_crossings(diff2)
            features.append(zeros_diff2)
            index.append(f'zeros_diff2_{c}')

            std_diff1 = np.std(diff1)
            features.append(std_diff1)
            index.append(f'std_diff1_{c}')

            std_diff2 = np.std(diff2)
            features.append(std_diff2)
            index.append(f'std_diff2_{c}')
            
            logging.debug('differential features generated')

            # Frequency features

            ## Use welch method to approcimate energies per frequency subdivision
            # From litterature the lowest frequencies of interest in a EEG is 0.5Hz so we need to keep our resolution at 0.25Hz hence a 4 second window cf.Nyquist
            window = (SAMPLING_FREQUENCY / DOWNSAMPLING_RATIO) * 4
            f, psd = scipy.signal.welch(data_channel, fs=80, nperseg=window)
            psd = np.nan_to_num(psd)

            ## Total summed energy
            channel_energy = band_energy(f, psd, 0.1, 40)
            features.append(channel_energy)
            index.append(f'channel_{c}_energy')

            ## Normalised summed energy
            normalised_energy = channel_energy / summed_energy
            features.append(normalised_energy)
            index.append(f'normalised_energy_{c}')

            ## Peak frequency
            peak_frequency = f[np.argmax(psd)]
            features.append(peak_frequency)
            index.append(f'peak_frequency_{c}')

            ## Normalised_summed energy per band
            for k in range(len(BANDS)-1):
                energy = band_energy(f, psd, BANDS[k], BANDS[k+1])
                normalised_band_energy = energy / channel_energy
                features.append(normalised_band_energy)
                index.append(f'normalised_band_energy_{c}_{k}')
                
            logging.debug('lowres frequency features generated')

            ## Spectral entropy
            psd_norm = np.divide(psd, psd.sum())
            spectral_entropy = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
            #spectral_entropy /= np.log2(psd_norm.size) #uncomment to normalise entropy
            features.append(spectral_entropy)
            index.append(f'spectral_entropy_{c}')

            ## SVD entropy
            entropy = svd_entropy(data_channel, order=3,
                                  delay=1, normalize=False)
            features.append(entropy)
            index.append(f'svd_entropy_{c}')
            
            logging.debug('entropy features generated')

            # Highres features : energy per frequency band in 1min segements
            highres_channel_energy = highres_total_energy(data)
            features.append(highres_channel_energy)
            index.append(f'total_channel_energy_{c}')
            
            f, psd = scipy.signal.welch(data, fs=400, nperseg=SAMPLING_FREQUENCY*4)
            psd = np.nan_to_num(psd)
            full_psd_sum = psd.sum()/10  # for normalisation purposed
            # TODO add band energy divided by full_psd_sum as feature
            
            # j allows us to iterate over 1min segments with 30s overlap
            for j in range(19):
                data_segment = data_channel[j*30*SAMPLING_FREQUENCY: (j+1)*30*SAMPLING_FREQUENCY]
                f_segment, psd_segment = scipy.signal.welch(
                    data_segment, fs=SAMPLING_FREQUENCY, nperseg=SAMPLING_FREQUENCY*4)
                psd_segment = np.nan_to_num(psd_segment)

                for k in range(len(HIGHRES_BANDS)-1):
                    window_band_energy = psd_segment[(f_segment > HIGHRES_BANDS[k]) & (
                        f_segment < HIGHRES_BANDS[k+1])].sum()
                    features.append(window_band_energy)
                    index.append(f'windowed_band_energy_{c}_{k}_{j}')
                    normalised_window_band_energy = window_band_energy/full_psd_sum
                    features.append(normalised_window_band_energy)
                    index.append(f'normalised_window_band_energy_{c}_{k}_{j}')
                    #TODO check if normalised feature is redundant
                    
            logging.debug('highres frequency features generated')
            
            #logging.debug(f'finished feature generation file:{counter}, channel:{c}')

        # Save generated features to X_train
        if counter == 0:
            #X_train = np.zeros((1, len(features)))
            X = np.array(features)
            logging.debug('X created and updated')   
        else:
            X = np.vstack((X, np.array(features)))
            logging.debug(f'features for file:{counter} added to array ; X.shape = {X.shape}')
        
        # Save label to y_train
        if is_training_data:
            label = filename[-5 : -4] #last char excluding .mat
            if counter == 0:
                y = np.array(label).astype('int')
                logging.debug(' y created and updated')
            else :
                y = np.vstack((y, np.array(label))).astype('int')
                logging.debug(f'label stacked onto y, y.shape = {y.shape}')
        
        counter += 1

        #TODO add logging

    # Save X_train to file before moving on to next patient data
    X = np.nan_to_num(X)
    y = np.nan_to_num(y)
       
    if is_training_data:
        if save_to_disk:
            np.save(join(FEATURE_SAVE_PATH, f'neurovista_X_train_pat{patient_number}.npy'), X)
            np.save(join(FEATURE_SAVE_PATH, f'neurovista_y_train_pat{patient_number}.npy'), y)
            logging.info('features and labels saved to disk')
        return (X, y)
    
    if is_training_data == False:
        if save_to_disk:
            np.save(join(FEATURE_SAVE_PATH, f'neurovista_X_test_pat{patient_number}.npy'), X)
            logging.info('features saved to disk')
        return X

#### Loop over our patients and call feature generation function

In [ ]:
data_dict = {}

for p in [1, 2, 3]:  #[1, 2, 3]:  # iterating over patients 1, 2, 3

    logging.info(f'Entering loop to generate train features for patient {p}')
    patient_path = join(DATA_PATH, TRAIN_PATHS[p-1])
    
    data_dict[f'X_train_pat{p}'], data_dict[f'y_train_pat{p}'] = generate_features(patient_number = p,
                                                                                   data_path = patient_path,
                                                                                   save_to_disk = True)

## EDA of the generated training dataset

In [ ]:
####

# II - Machine learning model

## Setup

### Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, f1_score
import sklearn.metrics
import logging
from sklearn.metrics import make_scorer
from evolutionary_search import EvolutionaryAlgorithmSearchCV #
from sklearn.model_selection import StratifiedKFold
from multiprocessing import Pool

C:\Users\gijsb\Anaconda3\envs\Epilepsy\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


### Metrics functions

In [2]:
def compute_metrics(clf, X_test, y_test):
    """
    return a dict containing auc, f1score, accuracy, balanced_accuracy, recall
    """
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)[:, [1]]
    
    auc = roc_auc_score(y_test, y_pred_proba)
    f1score = f1_score(y_test, y_pred)
    accuracy = sklearn.metrics.average_precision_score(y_test, y_pred)
    balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_test, y_pred, adjusted = True)
    recall = sklearn.metrics.recall_score(y_test, y_pred)
    
    metrics_dict = {}
    metrics_dict['auc'] = auc
    metrics_dict['f1score'] = f1score
    metrics_dict['accuracy'] = accuracy
    metrics_dict['balanced_accuracy'] = balanced_accuracy
    metrics_dict['recall'] = recall
    
    return metrics_dict

### Load datasets

In [ ]:
X_pat = {}
y_pat = {}

for p in [1,2,3]:
    X_pat[f'pat{p}'] = np.load(f'neurovista_X_train_pat{p}.npy').astype('float32')
    X_pat[f'pat{p}'] = np.nan_to_num(X_pat[f'pat{p}'])
    
    y_pat[f'pat{p}'] = np.load(f'neurovista_y_train_pat{p}.npy').astype('float32')
    y_pat[f'pat{p}'] = np.nan_to_num(y_pat[f'pat{p}'])
    
logging.debug('X and y loaded into dictionary')

# Assign data to train and test sets

X = np.vstack(tuple(X_pat.values()))
y = np.vstack(tuple(y_pat.values()))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)